In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [3]:
from epsampling.utils import drop_duplicate_cols

timestamp = '20241001-223952'

fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_{timestamp}.csv')
df_acs = pd.read_csv(fpath,index_col='Fips')

# Remove duplicate columns
df = drop_duplicate_cols(df_acs)
df

,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,EDU_PROF,HH_7pP,OCC_MBSA_CES_CM,...,POP_F,POP_AIANNH,POP_AIAN,EDU_HS,POP_A3564,POP_M,POP_16p_EMP_x7,POP_A25p,POP_NH,POP_x2
Fips,,,,,,,,,,,,,,,,,,,,,
2013,0,0,0,0,0,0,0,1,1,2,...,1395,1501,1505,1648,1760,2014,2213,2598,3050,3409
2016,0,6,9,11,12,47,180,14,21,17,...,2256,523,549,2626,2471,2995,3655,3838,4740,5251
2020,1842,2985,428,499,2642,2860,6892,5415,2311,3709,...,142897,19827,21150,94213,107273,149648,144598,193312,264762,292545
2050,32,90,250,239,9,19,150,102,543,31,...,8790,15427,15640,6485,5770,9724,6454,10025,18131,18514
2060,0,0,0,0,0,2,7,1,12,3,...,369,266,282,411,384,480,443,643,740,849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,165,3135,0,30,73,561,404,436,164,302,...,20609,454,506,16349,16838,21850,20884,27816,35626,42459
56039,209,67,0,74,97,175,404,496,0,179,...,11039,30,30,6148,9875,12280,14952,17659,19835,23319
56041,0,748,0,8,62,116,125,101,98,30,...,10077,20,52,8432,7709,10437,9688,13233,18533,20514


In [4]:
## Remove highly correlated features.

corr_thresh = 0.97

corr_matrix = df.corr().abs()
## Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
## Find features with correlation greater than like .99 ...
to_drop = [column for column in upper.columns if any(upper[column] > corr_thresh)]
## Drop features 
df_filt = df.drop(to_drop, axis=1, inplace=False)
df_filt

,OCC_MBSA_ELCAM_L,IND_AFFHM_MQE,IND_PSMAW_M,HU_OCC_OPRgt200,HU_UIS50P,IND_WHT_x7,HU_UIS2049,HH_7pP,OCC_MBSA_CES_CM,OCC_MBSA_HCPT_HTT,...,HU_VAC,IND_AFFHM,POP_OTH2PLNH,IND_PUBA_x7,POP_HISP_x12,POP_WHITENH,OCC_PTMM_P_x11,POP_ASIANNH,POP_AIANNH,POP_AIAN
Fips,,,,,,,,,,,,,,,,,,,,,
2013,0,0,0,0,0,0,0,1,2,3,...,199,216,330,339,359,436,710,712,1501,1505
2016,0,6,9,11,12,47,180,21,17,4,...,452,47,344,396,511,1200,739,2404,523,549
2020,1842,2985,428,499,2642,2860,6892,2311,3709,2189,...,11360,3545,29471,14462,27783,164211,3934,27758,19827,21150
2050,32,90,250,239,9,19,150,543,31,150,...,1472,148,532,1579,383,1703,275,257,15427,15640
2060,0,0,0,0,0,2,7,12,3,4,...,607,11,71,77,109,350,25,35,266,282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,165,3135,0,30,73,561,404,164,302,205,...,3645,3231,988,1295,6833,33243,1206,448,454,506
56039,209,67,0,74,97,175,404,0,179,124,...,3724,385,617,401,3484,18638,104,369,30,30
56041,0,748,0,8,62,116,125,98,30,173,...,1144,884,571,472,1981,17853,464,37,20,52
